In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from dateutil.relativedelta import *

import os
import re
from collections import Counter
from time import perf_counter

import pydot
import graphviz as gr
from networkx.drawing.nx_pydot import to_pydot
import networkx as nx
from IPython.display import Image, display

from dowhy import CausalModel

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.interpolate import interp1d

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from pmdarima.arima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.linear_model import LassoCV, LogisticRegressionCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import permutation_importance
from sklearn.base import clone

import dython
from dython.nominal import associations

from doubleml import DoubleMLData, DoubleMLPLR

from econml.dml import CausalForestDML

In [ ]:
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 3000)

In [ ]:
cur_dir = os.path.abspath(os.path.dirname("__fil__"))

# Data Aggregation and Cleaning

## OECD Files

In [ ]:
def OECD_clean_input(dataframe):
    dataframe = dataframe.astype(str).apply(lambda x: x.str.replace('"', ""))
    dataframe.rename(columns = lambda x: x.replace('"', ""), inplace=True)
    return dataframe

OECDCountryCodeSub = {"DEU":"DE", "FRA":"FR", "SWE":"SE", "ITA":"IT", "GBR":"UK", "DNK":"DK", 
                      "AUT":"AT", "HUN":"HU", "BEL":"BE", "FIN":"FI", "IRL":"IE", "NLD":"NL",
                      "NOR":"NO", "POL":"PL", "PRT":"PT", "CHE":"CH"}

### Projection Integration

#### Absolute Level of Projection is given

##### Total Population

In [ ]:
proj_pop_df = pd.read_csv(os.path.join(cur_dir,"ProjectedPopulation.csv"), quoting=3)
proj_pop_df = OECD_clean_input(proj_pop_df)
proj_pop_df["FREQUENCY"] = "A"

proj_pop_split_df = proj_pop_df[(proj_pop_df["Age"]=="Total") & (proj_pop_df["SEX"]=="T") & (proj_pop_df["TIME"]!="2022")]
proj_pop_split_df["Value"] = proj_pop_split_df["Value"].astype(float)/1000000

pop_df = pd.read_csv(os.path.join(cur_dir,"PopulationLevel.csv"), quoting=3)
pop_df = OECD_clean_input(pop_df)

proj_pop_combine_df = pd.concat([pop_df,proj_pop_split_df]).reset_index(drop=True)
proj_pop_combine_df["Country Code"] = proj_pop_combine_df["LOCATION"].map(OECDCountryCodeSub)

proj_pop_combine_df

##### Elderly Population

In [ ]:
proj_eldpop_split_df = proj_pop_df[(proj_pop_df["Age"]=="Share of 65 and over - elderly") & (proj_pop_df["SEX"]=="T") & (proj_pop_df["TIME"]!="2022")]
   
edlpop_df = pd.read_csv(os.path.join(cur_dir,"ElderlyPopulation.csv"), quoting=3)
edlpop_df = OECD_clean_input(edlpop_df)

proj_eldpop_combine_df = pd.concat([edlpop_df,proj_eldpop_split_df]).reset_index(drop=True)
proj_eldpop_combine_df["Country Code"] = proj_eldpop_combine_df["LOCATION"].map(OECDCountryCodeSub)

proj_eldpop_combine_df

#### Relative Level (Growth Rate) of Projection is given

##### GDP

In [ ]:
#Projection file to get years from 2023 and onwards

proj_gdp_df = pd.read_csv(os.path.join(cur_dir,"ProjectedGDPGrowthRate.csv"), quoting=3)
proj_gdp_df = OECD_clean_input(proj_gdp_df)
proj_gdp_df["Country Code"] = proj_gdp_df["LOCATION"].map(OECDCountryCodeSub)

proj_gdp_re_df = proj_gdp_df[proj_gdp_df["TIME"]!="2022"]

proj_gdp_re_df

In [ ]:
#Historical file to get only 2022 for to be used as base for growth projection

gdp_df = pd.read_csv(os.path.join(cur_dir,"GDP.csv"), quoting=3)
gdp_df = OECD_clean_input(gdp_df)
gdp_df["Country Code"] = gdp_df["LOCATION"].map(OECDCountryCodeSub)

gdp_re_df = gdp_df[gdp_df["TIME"]=='2022']

gdp_re_df

In [ ]:
#Combining historical and projection file with the first value (2022) being actual figure and the remaining figures as projection

proj_gdp_combine_df = pd.concat([gdp_re_df,proj_gdp_re_df]).sort_values(by=["Country Code","TIME"]).reset_index(drop=True)
proj_gdp_combine_df["Value"] = proj_gdp_combine_df["Value"].astype(float)

proj_gdp_combine_country_df =pd.DataFrame()

for country in OECDCountryCodeSub.values():
    country_single = proj_gdp_combine_df[proj_gdp_combine_df["Country Code"]==country].reset_index(drop=True)
    country_single.loc[0,"ProjectedGDPGrowth"] = country_single.loc[0,"Value"]
    for n in range(1, len(country_single)):
        previous = n-1
        country_single.loc[n,"ProjectedGDPGrowth"] = (1+(country_single.loc[n,"Value"]/100))*country_single.loc[previous,"ProjectedGDPGrowth"]
    proj_gdp_combine_country_df = pd.concat([proj_gdp_combine_country_df,country_single])
        
proj_gdp_combine_country_df = proj_gdp_combine_country_df.drop("Value",axis=1).rename(columns={"ProjectedGDPGrowth":"Value"})


#Add back the historical data before year 2022 for interpolation

gdp_re_hist_df = gdp_df[gdp_df["TIME"].astype(int)<2022]
proj_gdp_combine_country_df = pd.concat([gdp_re_hist_df,proj_gdp_combine_country_df]).reset_index(drop=True)
proj_gdp_combine_country_df

#### Time Series Projection

In [ ]:
#Define function for checking stationarity

def stationarity_check(country: [], file, x_axis_name: str):
    palette = sns.color_palette('Set2',len(country))
    country_color_dict = dict(zip(country, palette))
    
    if 'Market EUR' in file.columns:
        file["Country Code"] = file["Country"].map(country_zip)
    else:
        file = OECD_clean_input(file)
        file["Country Code"] = file["LOCATION"].map(OECDCountryCodeSub)

    file_time = file.copy()
    file_time['Value'] = file_time['Value'].astype(float)
    file_time['TIME'] = pd.to_datetime(file_time['TIME'], exact=False)
    file_time.set_index("TIME", inplace=True)
    file_time = file_time[["Country Code","Value"]]

    plt.figure(figsize=(10,5))

    for c in country:
        if 'Market EUR' in file.columns:
            file_time_country = file_time[file_time['Country Code']==c].resample('MS').asfreq().dropna()
        else:
            file_time_country = file_time[file_time['Country Code']==c].resample('YS').asfreq().dropna()
        adf, pvalue, usedlag_, nobs_, critical_values_, icbest_ = adfuller(file_time_country['Value'])
        
        if pvalue > 0.05:
            print('{} - Non-Stationary: {:.3f}'.format(c, pvalue))
        else:
            print('{} - Stationary: {:.3f}'.format(c, pvalue))

        sns.lineplot(x=file_time_country.index, y="Value", data=file_time_country, c=country_color_dict[c])

    plt.ylabel(ylabel=x_axis_name)
    plt.xlabel(xlabel='Year')
    legend_handles = [plt.Line2D([0,1],[1,1], label=c, color=country_color_dict[c], lw=2) for i, c in enumerate(country)]
    plt.legend(handles=legend_handles, title='Countries',loc='center left', fontsize=8, bbox_to_anchor=(1, 0.5))
    plt.tight_layout()
    plt.show()
    return file_time


In [ ]:
#Time Series Training and Prediction

def time_series_proj(country:[],proj_file:[[]],title_name:str):

    para_arima = np.arange(0,6,1)
    proj_file_proj = pd.DataFrame()
    fig,ax = plt.subplots(8,2,figsize=(16,30))

    for i, c in enumerate(country):

        #fix to yearly frequency
        proj_file_country = proj_file[proj_file['Country Code']==c].drop('Country Code', axis=1)
        
        freq_check = len(pd.date_range(proj_file_country.index[-2], proj_file_country.index[-1], freq='MS'))
        
        if freq_check==2:
            proj_file_country = proj_file_country.resample('MS').asfreq()
        elif freq_check==4:
            oidx = proj_file_country.index
            nidx = pd.date_range(oidx.min(), oidx.max(), freq='Q')
            proj_file_country = proj_file_country.reindex(oidx.union(nidx)).rename(columns={'Value':'Old Value'})
            proj_file_country['Value'] = proj_file_country['Old Value'].shift(-1)
            proj_file_country= proj_file_country.drop('Old Value',axis=1).dropna()
        else:
            proj_file_country = proj_file_country.resample('YS').asfreq()
            
        #check for optimal parameters
        arima_model = auto_arima(proj_file_country, start_p=para_arima[0], d=para_arima[0], start_q=para_arima[0],
                            max_p=para_arima[len(para_arima)-1], max_d=para_arima[len(para_arima)-1], max_q=para_arima[len(para_arima)-1],
                            start_P=para_arima[0], D=para_arima[0], start_Q=para_arima[0],
                            max_P=para_arima[len(para_arima)-1], max_D=para_arima[len(para_arima)-1], max_Q=para_arima[len(para_arima)-1],
                            stepwise=False)

        #split the data to train and test set
        size = int(len(proj_file_country)*0.8)
        X_train, X_test = proj_file_country[0:size], proj_file_country[size:len(proj_file_country)]

        #train the model with data training set
        if arima_model.get_params()['seasonal_order'][-1] > 1:
            model = SARIMAX(X_train, 
                            order=arima_model.get_params()['order'], 
                            seasonal_order=arima_model.get_params()['seasonal_order'],
                            initialization='approximate_diffuse')
        else:
            model = SARIMAX(X_train, 
                            order=arima_model.get_params()['order'],
                            initialization='approximate_diffuse')

        result = model.fit()
        
        
        #number of years to predict
        if freq_check==2:
            year_pred = len(pd.date_range((proj_file_country.index[-1]+relativedelta(months=1)),
                            pd.to_datetime('2024',format='%Y',exact=False),
                            freq='MS'))-1
                
        elif freq_check==4:
            year_pred = len(pd.date_range((proj_file_country.index[-1]+relativedelta(months=3)),
                            pd.to_datetime('2024',format='%Y',exact=False),
                            freq='QS'))-1

        else:
            year_pred = 2024-(proj_file_country.index[-1].year+1)
        

        #check the prediction of both training and testing set
        train_pred = result.predict(0,size)
        test_pred = result.predict(size, len(proj_file_country)+year_pred)
        

        #plot the prediction

        ax[i][0].plot(train_pred, label= 'Training Set Prediction')
        ax[i][0].plot(proj_file_country['Value'], label= 'Actual Figures')
        ax[i][0].plot(test_pred, label= 'Testing Set Prediction')
        ax[i][0].set_title('{} - Prediction with Splitting Dataset'.format(c))
        ax[i][0].legend()


        #train the model without splitting the data as seems like to more accurate for only predicting one year after
        #approximate_diffuse to avoid a possible error called LU Decomposition, not sure if there is any side-effect
        if arima_model.get_params()['seasonal_order'][-1] > 1:
            model = SARIMAX(proj_file_country, 
                            order=arima_model.get_params()['order'], 
                            seasonal_order=arima_model.get_params()['seasonal_order'],
                            initialization='approximate_diffuse')
        else:
            model = SARIMAX(proj_file_country, 
                            order=arima_model.get_params()['order'],
                            initialization='approximate_diffuse')

        result = model.fit()

        #check the prediction of both training and testing set
        train_pred = result.predict(0,size)
        test_pred = result.predict(size, len(proj_file_country)+year_pred)

        #plot the prediction

        ax[i][1].plot(train_pred, label= 'Training Set Prediction')
        ax[i][1].plot(proj_file_country['Value'], label= 'Actual Figures')
        ax[i][1].plot(test_pred, label= 'Testing Set Prediction')
        ax[i][1].set_title('{} - Prediction without Splitting Dataset'.format(c))
        ax[i][1].legend()

        #include the prediction value in the dataset
        pred_df = pd.DataFrame(test_pred[-(year_pred+1):]).rename(columns={'predicted_mean':'Value'})
        proj_file_country = pd.concat([proj_file_country,pred_df])
        proj_file_country['Country Code'] = c

        #concat all the individual back to one big table
        proj_file_proj = pd.concat([proj_file_proj,proj_file_country])

    plt.title(title_name)
    plt.tight_layout()
    plt.show()
    return proj_file_proj

## Disaggregation

In [ ]:
def proj_disagg(country:[], agg_fil:[[]], col_name:str):
    
    final_proj_disagg = pd.DataFrame()
    fig,ax = plt.subplots(2,4,figsize=(30,16))
    
    for i, c in enumerate(country):
        
        agg_fil_new = agg_fil.copy()
        
        if col_name in ts_proj:
            agg_fil_new = agg_fil_new.reset_index()
            agg_fil_new['TIME'] = pd.to_datetime(agg_fil_new['index'].astype(str)).dt.to_period('Y').dt.to_timestamp(how='end')
            
        else:
            agg_fil_new['TIME'] = pd.to_datetime(agg_fil_new['TIME']+'-12-31',exact=False)
            
        agg_fil_new = agg_fil_new[agg_fil_new['Country Code']==c].set_index('TIME')[['Value']]
        agg_fil_new['Value'] = agg_fil_new['Value'].astype(float)

        oidx = agg_fil_new.index
        nidx = pd.date_range(oidx.min(), oidx.max(), freq='M')
        res = agg_fil_new.reindex(oidx.union(nidx)).interpolate('cubicspline')
        
        res['Country Code'] = c
        final_proj_disagg = pd.concat([final_proj_disagg,res])
        
        ax.flatten()[i].plot(res['Value'], label= 'Disaggregated Level')
        ax.flatten()[i].plot(agg_fil_new['Value'], label= 'Actual Level')
        ax.flatten()[i].set_title('{} - {} Disaggregated Time Series'.format(c, col_name))
        ax.flatten()[i].legend()
        
    plt.show()
    display(final_proj_disagg)
    print('-'*122)
    
    final_proj_disagg = final_proj_disagg.reset_index().rename(columns={'Value':col_name})
    final_proj_disagg['Year'] = final_proj_disagg['index'].dt.year
    final_proj_disagg['Month'] = final_proj_disagg['index'].dt.month
        
    return final_proj_disagg

### Read all and combine all files

In [ ]:
folder_path = os.path.join(cur_dir,"OECD")

files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

PreAvoidDup = "1"
Agg_OECD_df = Agg_df.copy()

for file in files:
    file_path = os.path.join(folder_path, file)
    
    if file_path.lower().endswith("csv"):
        try:
            if 'actual' in file_path.lower():
                content = pd.read_csv(file_path, quoting=3)
                Agg_OECD_df = pd.merge(Agg_OECD_df, content.drop('index',axis=1), 
                                  on=["Country Code", "Year", "Month"], how="left")
                
            else:
                content = pd.read_csv(file_path, quoting=3)
                content = OECD_clean_input(content)
                content["Value"] = content["Value"].astype(float)
                content["Country Code"] = content["LOCATION"].map(OECDCountryCodeSub)
                content["Year_Month"] = content["TIME"]
                Agg_OECD_df = pd.merge(Agg_OECD_df, content[["Country Code", "Year_Month", "Value"]], 
                                  on=["Country Code", "Year_Month"], how="left")

                Agg_OECD_df.rename(columns={"Value":"Value"+PreAvoidDup}, inplace=True)

                PreAvoidDup += "1"            
                        
        except pd.errors.ParserError as e:
            print(f"Error parsing {file}: {e}")
            
    else:
        print(f"Skipping non-csv file: {file}")
        
OldColumnName = Agg_OECD_df.columns.tolist()
NewColumnName = ['Year_Month', 'Year', 'Month', 'Quarter', 'Country', 'Country Code', 'FiscalYear', 'FiscalMonth', 
                 'FiscalYearMonth', 'Average Regulatory Index Score', 'Household Disposable Income per Capita', 
                 'Elderly Population Proportion', 'GDP (in millions)', 'Healthcare Spending Proportion',
                 'Parmaceutical Spending Proportion', 'Population Level', 'Tertiary Education Level Proportion',
                 'Inflation Rate', 'Interest Rate', 'Unemployment Rate']
old_new_col_name_map = dict(zip(OldColumnName, NewColumnName))

Agg_OECD_df.rename(columns=old_new_col_name_map, inplace=True)

Agg_OECD_df["Elderly Population Proportion"] = Agg_OECD_df.apply(lambda df: df["Elderly Population Proportion"]*1000000 if df["Elderly Population Proportion"]<0.001
                                                      else df["Elderly Population Proportion"], axis=1)

Agg_OECD_df

## Financial Statement Data

In [ ]:
#Financial data function for reading IS and BS

def fin_data_read(folder_path:str, data_needed:str, first_file_toread:str, skiprows:int, usecols:[], filename:str,) -> pd.DataFrame:

    folder_path = folder_path

    files = [f for f in os.listdir(folder_path)]

    sheet_pattern = re.compile(r"^(?:Company_)?[A-Za-z]{{2}}{}$".format(data_needed))

    fin_df = pd.DataFrame({"YearMonth_Country_FinItems":["Year_Month","Country"]})
    sup_df = pd.read_excel(first_file_toread, 
                           sheet_name="Company{}".format(data_needed), skiprows=skiprows, usecols=[0])
    sup_df.columns = fin_df.columns
    fin_df = pd.concat([fin_df, sup_df], ignore_index=True)

    Avoid_Dup = "1"

    for file in files:
        file_path = os.path.join(folder_path, file)

        if file.upper().startswith(filename):
            file_ext_name = os.path.basename(file_path)
            file_year_month = file_ext_name[-9:-5]

            file_df = pd.ExcelFile(file_path)
            all_sheet_names = file_df.sheet_names

            for sheet_name in all_sheet_names:
                if sheet_name != "Company_BS":
                    if sheet_pattern.match(sheet_name):
                        file_country = sheet_name
                        sup_data = file_df.parse(sheet_name, skiprows=skiprows, usecols=usecols)
                        sup_data.columns = ["YearMonth_Country_FinItems","YearMonth_Country_FinData"+Avoid_Dup]

                        month_data = pd.DataFrame({"YearMonth_Country_FinItems":["Year_Month","Country"],
                                                  "YearMonth_Country_FinData"+Avoid_Dup:[file_year_month,file_country]})

                        fin1_df = pd.concat([month_data,sup_data], ignore_index=True)
                        try:
                            fin1_df.iloc[9,0] = fin1_df.iloc[9,0].replace("Payments","Payments_sub")

                        except:
                            pass
                        fin_df = pd.merge(fin_df, fin1_df, on="YearMonth_Country_FinItems", how="outer")
                        fin_df = fin_df.dropna(subset=["YearMonth_Country_FinItems"], how="all")

                        Avoid_Dup += "1"

    return fin_df


## Datahub

#### Price Range Categorization

In [ ]:
price_quantiles = pd.qcut(outbound_excl_cus["OrderPriceSales"], q=[0,0.2,0.4,0.6,0.8,1.0], labels=["Very Low","Low", "Medium", "High", "Very High"])

outbound_price_range = outbound_excl_cus.loc[:]
outbound_price_range["Price Categorization"] = price_quantiles

outbound_price_range = outbound_price_range.groupby(["Year_Month","Country Code","Price Categorization"])["Deliver Export Turnover"].sum().reset_index()
outbound_tot_price = outbound_price_range.groupby(["Year_Month","Country Code"])["Deliver Export Turnover"].transform("sum")

outbound_price_range["Price Range Revenue Proportion"] = outbound_price_range["Deliver Export Turnover"]/outbound_tot_price

outbound_price_range_pivot = outbound_price_range.drop("Deliver Export Turnover", axis=1).pivot_table(index=["Year_Month","Country Code"],
                                                                                                     columns =["Price Categorization"],
                                                                                                     values=["Price Range Revenue Proportion"],
                                                                                                     aggfunc="sum",
                                                                                                     fill_value=0)

outbound_price_range_pivot.columns = [f"{col[1]}_{col[0]}" for col in outbound_price_range_pivot.columns]
outbound_price_range_pivot.reset_index(inplace=True)

outbound_price_range_pivot

# Exploratory Data Analysis

### All Features

#### PCA Analysis

In [ ]:
num_transformer = Pipeline(steps=[('NaNnum',SimpleImputer(strategy='mean')),
                                  ('scale',StandardScaler())])

bi_cat_transformer = Pipeline(steps=[('NaNcat',SimpleImputer(strategy='most_frequent')),
                                  ('onehot',OneHotEncoder())])

preprocessor_pca = ColumnTransformer(transformers=[('num_transformer',num_transformer,target_features+numeric_features),
                                               ('bi_cat_transformer',bi_cat_transformer,binary_categorical_features)])

pipeline_pca = Pipeline(steps=[('preprocessor',preprocessor_pca),
                           ('pca',PCA(n_components=2))])

pca = pipeline_pca.fit_transform(Agg_df_infadj[target_features+numeric_features+binary_categorical_features])

In [ ]:
palette = sns.color_palette('Set2',len(country))
country_color_dict = dict(zip(country,palette))
country_color = [country_color_dict[x] for x in Agg_df_infadj["Country"]]

for sample in range(len(pca)):
    plt.scatter(pca[sample][0],pca[sample][1],color=country_color[sample],s=8)
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.title('PCA Analysis')
legend_handles = [plt.Line2D([0],[0], marker='o', color='w', label=label,
                             markerfacecolor=palette[i], markersize=8) for i, label in enumerate(country)]
plt.legend(handles=legend_handles, title='Countries',loc='center left', fontsize=8, bbox_to_anchor=(1, 0.5))
plt.show()

#### Numeric Features

In [ ]:
#Summary statistics for numeric features

summary_stat_num = Agg_df_infadj[target_features+numeric_features].describe()
summary_stat_num.loc["+3_std"] = summary_stat_num.loc["mean"]+(summary_stat_num.loc["std"]*3)
summary_stat_num.loc["-3_std"] = summary_stat_num.loc["mean"]-(summary_stat_num.loc["std"]*3)

summary_stat_num

In [ ]:
#Possible outliers outside of +/-3 std
pos_out_num = Agg_df_infadj[(np.abs(stats.zscore(Agg_df_infadj[target_features+numeric_features]))>3).any(axis=1)]
pos_out_num

In [ ]:
#Correlation Matrix
corr_num = Agg_df_infadj[target_features+numeric_features].corr()

plt.figure(figsize=(15,15))
sns.heatmap(corr_num, xticklabels=corr_num.columns, yticklabels=corr_num.columns, cmap="RdBu")
plt.show()

display(corr_num)

In [ ]:
pd.plotting.scatter_matrix(corr_num, alpha=1, figsize=(100,100))

#### Categorical Features

In [ ]:
summary_stat_bi_cat = Agg_df_infadj[binary_categorical_features].describe(include=["category","object"])

summary_stat_bi_cat

In [ ]:
#from github (dython)
associations(Agg_df_infadj[binary_categorical_features], nom_nom_assoc="theil", figsize=(15, 15), cmap='RdBu')["corr"]


## Main Features

In [ ]:
#from github (dython)
associations(Agg_df_infadj[target_features+main_numeric_features+main_binary_categorical_features], nom_num_assoc="correlation_ratio", figsize=(15, 15), cmap='RdBu')['corr']


### Univariate Analysis

#### Numeric main Features

In [ ]:
#Not transformed with log

fig, axes = plt.subplots(6, 4, figsize=(25,20))

for i, ax in enumerate(axes.flatten()):
    if i < len(target_features+main_numeric_features):
        sns.histplot(x=Agg_df_infadj[target_features+main_numeric_features].iloc[:,i], ax=ax)
        ax.axvline(Agg_df_infadj[target_features+main_numeric_features].iloc[:,i].mean(), color='magenta', linestyle='dashed', linewidth=2)
        ax.axvline(Agg_df_infadj[target_features+main_numeric_features].iloc[:,i].median(), color='cyan', linestyle='dashed', linewidth=2)
        ax.set_title(Agg_df_infadj[target_features+main_numeric_features].columns[i])
        ax.set_xlabel(xlabel='')
        ax.set_ylabel(ylabel='')
    else:
        ax.axis('off')

fig.suptitle("Univariate Analysis of Key Numeric Features", fontsize='x-large')       
        
plt.show()

In [ ]:
#Transformed with log

fig, axes = plt.subplots(6, 4, figsize=(25,20))

for i, ax in enumerate(axes.flatten()):
    if i < len(target_features+main_numeric_features):
        sns.histplot(x=np.log1p(Agg_df_infadj[target_features+main_numeric_features]).iloc[:,i], ax=ax)
        ax.axvline(np.log1p(Agg_df_infadj[target_features+main_numeric_features]).iloc[:,i].mean(), color='magenta', linestyle='dashed', linewidth=2)
        ax.axvline(np.log1p(Agg_df_infadj[target_features+main_numeric_features]).iloc[:,i].median(), color='cyan', linestyle='dashed', linewidth=2)
        ax.set_title(Agg_df_infadj[target_features+main_numeric_features].columns[i])
        ax.set_xlabel(xlabel='')
        ax.set_ylabel(ylabel='')
    else:
        ax.axis('off')

fig.suptitle("Univariate Analysis of Key Numeric Features (Log Transformed)", fontsize='x-large')       
        
plt.show()

#### Categorical main Features

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(10,10), gridspec_kw={'hspace':0.6})

for i, ax in enumerate(axes.flatten()):
    if i < 2:
        sns.countplot(x=Agg_df_infadj[main_binary_categorical_features].iloc[:,i], ax=ax, palette='Blues')
        ax.set_title(Agg_df_infadj[main_binary_categorical_features].columns[i])
        ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
        ax.set_xlabel(xlabel='')
        ax.set_ylabel(ylabel='')
    else:
        sns.countplot(x=Agg_df_infadj[main_binary_categorical_features].iloc[:,i], ax=ax, palette='Blues')
        ax.set_title(Agg_df_infadj[main_binary_categorical_features].columns[i])
        ax.set_xlabel(xlabel='')
        ax.set_ylabel(ylabel='')
    
fig.suptitle("Univariate Analysis of Key Binary and Categorical Features",fontsize='x-large')

plt.tight_layout()
plt.show()

### Bivariate Analysis

#### Numeric main Features

In [ ]:
sns.pairplot(Agg_df_infadj, vars=target_features+main_numeric_features, hue='Country', corner=True, palette='Set2')

#### Categorical main Features

In [ ]:
fig, axes = plt.subplots(2,4,figsize=(35,15),sharey='row', sharex='col')


for t_i, target in enumerate(target_features):
    for p_i, pred in enumerate(main_binary_categorical_features):
        if t_i==0 and p_i==0:
            ax=axes[t_i][p_i]
            sns.boxplot(Agg_df_infadj,
                x=pred, 
                y=target, 
                hue="Country", 
                palette='Set2',
                ax=ax)
            ax.set_xlabel(xlabel='')
            handles, labels = ax.get_legend_handles_labels()
            ax.get_legend().remove()
            
        elif t_i==1 and p_i==0:
            ax=axes[t_i][p_i]
            sns.boxplot(Agg_df_infadj,
                x=pred, 
                y=target, 
                hue="Country", 
                palette='Set2',
                ax=ax)
            ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
            ax.get_legend().remove()
            
        elif t_i==1 and p_i in range(1,len(main_binary_categorical_features)):
            ax=axes[t_i][p_i]
            sns.boxplot(Agg_df_infadj,
                x=pred, 
                y=target, 
                hue="Country", 
                palette='Set2',
                ax=ax)
            if p_i == 1:
                ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
            ax.set_ylabel(ylabel='')
            ax.get_legend().remove()
        
        else:
            ax=axes[t_i][p_i]
            sns.boxplot(Agg_df_infadj,
                x=pred, 
                y=target, 
                hue="Country", 
                palette='Set2',
                ax=ax)
            ax.set_xlabel(xlabel='')
            ax.set_ylabel(ylabel='')
            ax.get_legend().remove()
            
fig.legend(handles,labels,loc='outside right center', title="Country")

plt.tight_layout()
plt.show()

## Feature Importance (Random Forest)

In [ ]:
####Train the model

folds = 5
cv = KFold(n_splits=folds-1)
param_grid = {'rf__max_depth': [10,20,30,40,50],
              'rf__n_estimators': np.arange(50,350,50)}

preprocessor_rf = ColumnTransformer(transformers=[('num_transformer',num_transformer,numeric_features),
                                               ('bi_cat_transformer',bi_cat_transformer,binary_categorical_features)])

pipeline_rf = Pipeline(steps=[('preprocessor',preprocessor_rf),
                           ('rf',RandomForestRegressor(n_estimators=100, random_state=0))])


GSCV_rf = GridSearchCV(estimator=pipeline_rf, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=cv)

X = Agg_df_infadj[numeric_features+binary_categorical_features]

for i in target_features:
    y = SimpleImputer(strategy='mean').fit_transform(Agg_df_infadj[[i]]).reshape(-1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/folds, random_state=0)

    GSCV_rf.fit(X_train,y_train)

    
    ####Model Evaluation and Prediction

    print('GridSearchCV Random Forest NMAE Best Score for Target Feature - {}: {:.3f}'.format(i,GSCV_rf.best_score_))
    print('GridSearchCV Random Forest NMAE Best Score for Target Feature - {}: {}'.format(i,GSCV_rf.best_params_))
    print('-'*100)

    y_pred = GSCV_rf.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print('MSE - {}: {:.3f}'.format(i,mse))
    print('RMSE - {}: {:.3f}'.format(i,rmse))
    print('R2 - {}: {:.3f}'.format(i,r2))
    print('-'*100)

    plt.scatter(y_test, y_pred)
    plt.xlabel('Actual Value')
    plt.ylabel('Predicted Value')
    plt.title('Random Forest Predictions (All Features) - {}'.format(i))
    coeff = np.polyfit(y_test,y_pred,1)
    p_line = np.poly1d(coeff)
    plt.plot(y_test,p_line(y_test),color='darkorange')

    plt.show()

    
    ####Feature Importance (MDI - Impurity)

    rf_feat_name = [name.split('__')[1] for name in GSCV_rf.best_estimator_[:-1].get_feature_names_out()]
    rf_feat_imp = GSCV_rf.best_estimator_[-1].feature_importances_

    feat_imp_df = pd.Series(rf_feat_imp, index=rf_feat_name).sort_values(ascending=False)

    plt.figure(figsize=(20,20))
    sns.barplot(y=feat_imp_df.index,x=feat_imp_df,palette='Blues_r')
    plt.title('Random Forest Feature Importance (MDI) - {}'.format(i),fontsize='xx-large')
    plt.tight_layout()
    plt.show()
    
    
    ####Feature Importance (MDA - Accurcy - Permutation)
    
    #On training data
    feat_MDA_result = permutation_importance(
        GSCV_rf, X_train, y_train, n_repeats=10, random_state=0, n_jobs=2)

    sorted_importance_idx = feat_MDA_result.importances_mean.argsort()

    feat_imp_pi_df = pd.DataFrame(feat_MDA_result.importances[sorted_importance_idx].T,
                                  columns=X.columns)

    plt.figure(figsize=(20,20))
    sns.boxplot(feat_imp_pi_df,orient='h',palette='Blues')
    plt.axvline(x=0, color="darkorange", linestyle="--", linewidth=2)
    plt.title('Random Forest Feature Importance_Training Set (MDA) - {}'.format(i),fontsize='xx-large')
    plt.tight_layout()
    plt.show()
    
    
    #On testing data
    feat_MDA_result = permutation_importance(
        GSCV_rf, X_test, y_test, n_repeats=10, random_state=0, n_jobs=2)

    sorted_importance_idx = feat_MDA_result.importances_mean.argsort()

    feat_imp_pi_df = pd.DataFrame(feat_MDA_result.importances[sorted_importance_idx].T,
                                  columns=X.columns)

    plt.figure(figsize=(20,20))
    sns.boxplot(feat_imp_pi_df,orient='h',palette='Blues')
    plt.axvline(x=0, color="darkorange", linestyle="--", linewidth=2)
    plt.title('Random Forest Feature Importance_Test Set (MDA) - {}'.format(i),fontsize='xx-large')
    plt.tight_layout()
    plt.show()

# DAG Analysis

In [ ]:
#DAG Basic Examples

DAG_chain = gr.Digraph()
DAG_chain.edge("X", "T")
DAG_chain.edge("T", "Y")
DAG_chain

DAG_fork = gr.Digraph()
DAG_fork.edge("X", "T")
DAG_fork.edge("X", "Y")
DAG_fork

DAG_collider = gr.Digraph()
DAG_collider.edge("X", "Y")
DAG_collider.edge("T", "Y")
DAG_collider

DAG_chain.render("Chain_DAG",view=True,format="png")
DAG_fork.render("Fork_DAG",view=True,format="png")
DAG_collider.render("Collider_DAG",view=True,format="png")

#Backdoor Adjustment Example

DAG_b4Backdoor = gr.Digraph()
DAG_b4Backdoor.edge("X", "T")
DAG_b4Backdoor.edge("X", "Y")
DAG_b4Backdoor.edge("T", "Y")
DAG_b4Backdoor

DAG_afterBackdoor = gr.Digraph()
DAG_afterBackdoor.edge("X", "Y")
DAG_afterBackdoor.edge("T", "Y")
DAG_afterBackdoor

DAG_b4Backdoor.render("b4Backdoor_DAG",view=True,format="png")
DAG_afterBackdoor.render("afterBackdoor_DAG",view=True,format="png")

In [ ]:
#Update the macro, industry and micro features after the selection above for building official final DAG for the model

macro_features_adj = [col for col in macro_features if col in numeric_features + binary_categorical_features]
industry_features_adj = [col for col in industry_features if col in numeric_features + binary_categorical_features]
micro_features_adj = [col for col in micro_features if col in numeric_features + binary_categorical_features]

#df for final DAG
df_graph = df_EDA[target_feature + country_indicator + time_indicator + macro_features_adj + 
                  industry_features_adj + micro_features_adj]
df_graph

In [ ]:
#edges for final DAG for the model

edges_to_EDITDA = [(c, 'EBITDA_Thousands') for c in df_graph.iloc[:, 1: len(df_graph.columns)].columns]

edges_from_coun_time_to_macro = [(c, o)
                                 for c in df_graph.iloc[:, 1: df_graph.columns.get_loc('GDP_Millions')].columns
                                 for o in df_graph.iloc[:, df_graph.columns.get_loc('GDP_Millions'): df_graph.columns.get_loc('RegulatoryIndex')].columns]
    
edges_from_macro = [(c, o) 
                    for c in df_graph.iloc[:, 1: df_graph.columns.get_loc('RegulatoryIndex')].columns
                    for o in df_graph.iloc[:, df_graph.columns.get_loc('RegulatoryIndex'): len(df_graph.columns)].columns]

edges_from_Reg = [('RegulatoryIndex', o) 
                  for o in df_graph.iloc[:, df_graph.columns.get_loc('RegulatoryIndex') + 1: len(df_graph.columns)].columns]

edges_from_Pharm = [('PharmacyDensity_Per100k', o) 
                  for o in df_graph.iloc[:, df_graph.columns.get_loc('PharmacyDensity_Per100k') + 1: len(df_graph.columns)].columns]

edges_to_MarketShare_p1 = [(c, 'Market_EUR') 
                           for c in df_graph.iloc[:, df_graph.columns.get_loc('RegulatoryIndex'): df_graph.columns.get_loc('Market_EUR')].columns]
edges_to_MarketShare_p2 = [(c, 'Market_EUR') 
                           for c in df_graph.iloc[:, df_graph.columns.get_loc('Market_EUR') + 1: len(df_graph.columns)].columns]

other_edges = [('GDP_Millions', 'HealthcareSpendProp'),
               ('GDP_Millions', 'UnemploymentRate'),
               ('PopulationLevel_Millions', 'HealthcareSpendProp'),
               ('PopulationLevel_Millions', 'UnemploymentRate'),
               ('NumWholesaleWarehouses', 'StockDays'),
               ('NumDistinctCustomersServed', 'StockDays'),
               ('OutboundServiceLevel', 'StockDays'),
               ('NumDistinctCustomersServed', 'DebtorDays'),
               ('NumDistinctSuppliersDealtWith', 'StockDays'),
               ('InboundServiceLevel', 'StockDays'),
               ('NumDistinctSuppliersDealtWith', 'CreditorDays')]

In [ ]:
causal_graph = nx.DiGraph(
    edges_to_EDITDA +
    edges_from_coun_time_to_macro +
    edges_from_macro +
    edges_from_Reg + 
    edges_from_Pharm +
    edges_to_MarketShare_p1 + 
    edges_to_MarketShare_p2 +
    other_edges
)

nx.draw(causal_graph, 
        with_labels=True,
        node_size=2000,
        font_size=10)

In [ ]:
#Put the DAG to dowhy and identify backdoor path for each treatment

backdoor_path = {}

for col in main_numeric_features + main_binary_categorical_features:

    model = CausalModel(
       data=df_graph,
       treatment=col,
       outcome='EBITDA_Thousands',
       graph="\n".join(nx.generate_gml(causal_graph))
    )

    identified_estimand = model.identify_effect(
        proceed_when_unidentifiable=True,
        method_name='maximal-adjustment'
    )

    print('Backdoor length: ', len(identified_estimand.get_backdoor_variables()))
    print('Backdoor variables: ', identified_estimand.get_backdoor_variables())
    print(identified_estimand)
    print('-'*100)

    backdoor_path[col] = identified_estimand.get_backdoor_variables()

In [ ]:
#Excluded variables in backdoor path

for col in main_numeric_features + main_binary_categorical_features:
    path = backdoor_path[col]
    notback = dict()
    notback[col] = [c for c in country_indicator+time_indicator+numeric_features+binary_categorical_features if c not in path+[col]]
    print(notback)
    print(len(notback[col]))
    
notback 

## Linear Regression

In [ ]:
#Just dummy and clustering for OLS 

df_linear_dummy = pd.get_dummies(df_EDA[country_indicator+time_indicator+binary_categorical_features], 
                                 drop_first=True, dtype=float).drop('Country_Sweden', axis=1)
df_linear_num = df_EDA[target_feature+numeric_features]

df_linear = pd.concat([df_linear_num, df_linear_dummy], axis=1)

In [ ]:
model = sm.OLS(df_linear['EBITDA_Thousands'], 
               sm.add_constant(df_linear.loc[:, df_linear.columns != 'EBITDA_Thousands'])).fit()

model.summary()

In [ ]:
ols_result_df = pd.DataFrame()

ols_result_df = pd.concat([ols_result_df, model.params], axis=1).rename(columns={0: 'Coefficient'})
ols_result_df = pd.concat([ols_result_df, model.bse], axis=1).rename(columns={0: 'Std. Error'})
ols_result_df = pd.concat([ols_result_df, model.pvalues], axis=1).rename(columns={0: 'P-value'})
ols_result_df = pd.concat([ols_result_df, model.conf_int()], axis=1).rename(columns={0: 'CI Lower Bound (2.5%)', 1: 'CI Upper Bound (97.5%)'})

ols_result_df.to_excel('ols_result.xlsx')

ols_result_df

In [ ]:
ols_result_df = pd.read_excel('ols_result.xlsx')

for col in ols_result_df.columns:
    if col != 'Unnamed: 0':
        if col != 'P-value':
            ols_result_df[col] = ols_result_df[col].round(2)

        else:
            ols_result_df[col] = ols_result_df[col].round(3)
        
ols_result_df['Method'] = 'OLS'
        
ols_result_df = ols_result_df.rename(columns={'Unnamed: 0':'Feature'})
ols_result_df

## Causal Machine Learning

In [ ]:
#transform the data with standard scaler, poly and one-hot encoding (poly only happens to Lasso)

def sc_poly_ohe_preprocess(df, ml_type, column):
    
    #treatment variable specific backdoor path
    backdoor = backdoor_path[column]  
    #num col adjusted for both treatment and backdoor variables 
    num_col_no_treat = [col for col in numeric_features if col in backdoor]
    #cat col adjusted for both treatment and backdoor variables 
    cat_col_no_treat = [col for col in binary_categorical_features if col in backdoor]
    #country col adjusted for both treatment and backdoor variables
    adj_country_indicator = [col for col in country_indicator if col in backdoor]
    #time col adjusted for both treatment and backdoor variables
    adj_time_indicator = [col for col in time_indicator if col in backdoor]
    
    #StandardScaler transform
    sc = StandardScaler().set_output(transform='pandas')
    df_sc = sc.fit_transform(df[num_col_no_treat])

    if ml_type == 'Lasso':
        #poly no interaction transform
        df_poly_num2 = np.power(df_sc, 2).rename(columns=lambda x: x+'_2nd')
        df_poly_num2 = df_poly_num2 - df_poly_num2.mean(axis=0)
        df_poly_num3 = np.power(df_sc, 3).rename(columns=lambda x: x+'_3rd')
        df_poly_num3 = df_poly_num3 - df_poly_num3.mean(axis=0)
        df_poly_all = pd.concat([df_sc, df_poly_num2, df_poly_num3], axis=1)
    
        #dummy transform
        df_dummy = pd.get_dummies(df[adj_country_indicator+adj_time_indicator+cat_col_no_treat], dtype=float).drop('Country_Sweden', axis=1)
        df_poly_dummy = df_poly_all.join(df_dummy)
        
        #create interaction terms
        poly = PolynomialFeatures(2, interaction_only=True, include_bias=False)
        df_poly_dummy_interact = poly.fit_transform(df_poly_dummy)
        poly_cols = poly.get_feature_names_out(df_poly_dummy.columns)
        df_poly_dummy_interact = pd.DataFrame(df_poly_dummy_interact, columns=poly_cols)
        
        filtered_binary = df_poly_dummy_interact.columns[df_poly_dummy_interact.nunique()<=2]

        #All the duplicated categorical variables interaction
        duplicate_interact_same_var = [col for col in filtered_binary if col.split('_')[0] in col.split('_')[1]]

        ##Drop the duplicated interacting variables (e.g. RegulatoryIndex_LowImpact X RegulatoryIndex_HighImpact)
        #All the categorical variables and categorical variables with interaction
        df_poly_dummy_interact = df_poly_dummy_interact.drop(duplicate_interact_same_var, axis=1)

        df_final = pd.concat([df[target_feature+[column]], df_poly_dummy_interact], axis=1)

    else:
        #dummy transform
        df_dummy = pd.get_dummies(df[adj_country_indicator+adj_time_indicator+cat_col_no_treat], dtype=float).drop('Country_Sweden', axis=1)
        
        df_final = pd.concat([df[target_feature+[column]], df_sc, df_dummy], axis=1)
    
    #Shuffle the dataset
    df_final = df_final.sample(frac=1, random_state=0).reset_index(drop=True)
    
    return df_final

In [ ]:
#Helper function for running all the treatment in one go in different CML models 

def cml_implement(df, ml_type, cml_type, sensitivity=False):
    
    cml_final_result = pd.DataFrame()
    
    if cml_type == 'dml':
        cml_final_score = {'ml_y': {}, 'ml_t': {}}
        
    else:
        cml_final_score = {'yhat_that': {}, 'ml_y': {}, 'ml_t': {}}
    
    for feature in main_numeric_features + main_binary_categorical_features:      

        t_2_start = perf_counter()
        
        if cml_type == 'dml':
            if feature in main_numeric_features:
                result_all, score_dict, _ = dml_wrapper(df, ml_type, 'num', feature)

            else:
                result_all, score_dict, _ = dml_wrapper(df, ml_type, 'cat', feature)     
        
        else:
            if feature in main_numeric_features:
                result_all, score_dict, _ = cf_wrapper(df, ml_type, 'num', feature)

            else:
                result_all, score_dict, _ = cf_wrapper(df, ml_type, 'cat', feature)    

        t_2_stop = perf_counter()

        print(result_all)
        print(score_dict)
        print('Time used this round: {:.4f} seconds'.format(t_2_stop - t_2_start))
        print('-'*100)  

        cml_final_result = pd.concat([cml_final_result, result_all])
        
        if cml_type != 'dml':
            cml_final_score['yhat_that'].update(score_dict['yhat_that'])
            
        cml_final_score['ml_y'].update(score_dict['ml_y'])
        cml_final_score['ml_t'].update(score_dict['ml_t'])
    
    return (cml_final_result, cml_final_score)

### DML

In [ ]:
#Produce dml output for a single feature

def dml_wrapper(df, ml_type, data_type, column):
    
    #penalty parameter for tuning logistic
    Cs = 0.0001*np.logspace(0, 4, 10)
    
    #parameters for tuning RF
    max_features = ['sqrt', 'log2']
    max_depth = [3, 5, 7, 9]
    min_samples_split = [10, 15, 20]
    min_samples_leaf = [5, 10, 15]
    n_estimators = np.arange(500,1000,100)
    
    par_grids_rf = {'ml_l': {'max_depth': max_depth,
                             'max_features': max_features,
                             'min_samples_split': min_samples_split,
                             'min_samples_leaf': min_samples_leaf,
                             'n_estimators': n_estimators},
                    'ml_m': {'max_depth': max_depth,
                             'max_features': max_features,
                             'min_samples_split': min_samples_split,
                             'min_samples_leaf': min_samples_leaf,
                             'n_estimators': n_estimators}}
    
    df_processed = sc_poly_ohe_preprocess(df, ml_type, column)
    
    nuisance_score_dict = {'ml_y': {}, 'ml_t': {}}

    if data_type == 'num':
        dml_data = DoubleMLData(df_processed,
                                y_col = target_feature[0],
                                d_cols = column,
                                x_cols = df_processed.iloc[:, 2:].columns.tolist())
        
        np.random.seed(22)
        
        if ml_type == 'Lasso':
            ml_l = clone(LassoCV(max_iter=10000, random_state=22))
            ml_m = clone(LassoCV(max_iter=10000, random_state=22))

            obj_dml_plr = DoubleMLPLR(dml_data, ml_l=ml_l, ml_m=ml_m)

        else: 
            ml_l = clone(RandomForestRegressor(random_state=22))
            ml_m = clone(RandomForestRegressor(random_state=22))

            obj_dml_plr = DoubleMLPLR(dml_data, ml_l=ml_l, ml_m=ml_m)
            obj_dml_plr.tune(par_grids_rf, search_mode='grid_search', n_jobs_cv=-1)

        obj_dml_plr.fit(n_jobs_cv=-1)
        nuisance_score_dict['ml_y'][column] = obj_dml_plr.evaluate_learners()['ml_l'].item()
        nuisance_score_dict['ml_t'][column] = obj_dml_plr.evaluate_learners()['ml_m'].item()
        result_all = obj_dml_plr.summary              

    else:
        result_all = pd.DataFrame()

        #to get the dummies of the targeted column and run the model of that one by one (except the first one that is dropped)
        bi_cat_col = pd.DataFrame(df_processed.pop(column))
        bi_cat_dum = pd.get_dummies(bi_cat_col[[column]], drop_first=True, dtype=float)
        df_processed = pd.concat([bi_cat_dum, df_processed], axis=1)

        #to calculate for x_cols
        bin_cat_dum_num = len(bi_cat_dum.columns)

        for column in bi_cat_dum.columns:
            dml_data = DoubleMLData(df_processed,
                                    y_col = target_feature[0],
                                    d_cols = column,
                                    x_cols = df_processed.iloc[:, 1+bin_cat_dum_num:].columns.tolist()) 
            
            np.random.seed(22)
            
            if ml_type == 'Lasso':
                ml_l = LassoCV(max_iter=10000, random_state=22)
                ml_m = LogisticRegressionCV(penalty='l1', solver='liblinear', Cs=Cs, max_iter=10000, random_state=22)

                obj_dml_plr = DoubleMLPLR(dml_data, ml_l=ml_l, ml_m=ml_m)

            else:
                ml_l = RandomForestRegressor(random_state=22)
                ml_m = RandomForestClassifier(random_state=22)

                obj_dml_plr = DoubleMLPLR(dml_data, ml_l=ml_l, ml_m=ml_m)
                obj_dml_plr.tune(par_grids_rf, search_mode='grid_search', n_jobs_cv=-1)

            obj_dml_plr.fit(n_jobs_cv=-1)
            nuisance_score_dict['ml_y'][column] = obj_dml_plr.evaluate_learners()['ml_l'].reshape(-1).tolist()
            nuisance_score_dict['ml_t'][column] = obj_dml_plr.evaluate_learners()['ml_m'].reshape(-1).tolist()
            result = obj_dml_plr.summary

            result_all = pd.concat([result_all, result])

    return (result_all, nuisance_score_dict, obj_dml_plr)
        

#### Lasso

In [ ]:
result_all, _, _ = dml_wrapper(df_EDA, 'Lasso', "cat", "RegulatoryIndex")
result_all

In [ ]:
lasso_dml_final_result, lasso_dml_nuisance_final_score = cml_implement(df_EDA, 'Lasso', 'dml')

In [ ]:
lasso_dml_final_result.to_excel('lasso_dml_final_result.xlsx')

lasso_dml_final_result

In [ ]:
#Transform the results with proper titles

lasso_dml_final_result_adj = pd.read_excel('lasso_dml_final_result.xlsx')

lasso_dml_final_result_adj = lasso_dml_final_result_adj.rename(columns={'Unnamed: 0':'Feature', 'coef':'Coefficient',
                                                                        'std err':'Std. Error', 'P>|t|': 'P-value',
                                                                        '2.5 %':'CI Lower Bound (2.5%)', '97.5 %':'CI Upper Bound (97.5%)'}).drop('t', axis=1)

for col in lasso_dml_final_result_adj.columns:
    if col != 'Feature':
        if col != 'P-value':
            lasso_dml_final_result_adj[col] = lasso_dml_final_result_adj[col].round(2)

        else:
            lasso_dml_final_result_adj[col] = lasso_dml_final_result_adj[col].round(3)
        
lasso_dml_final_result_adj['Method'] = 'DML - Lasso'

lasso_dml_final_result_adj

#### Random Forest

In [ ]:
rf_dml_final_result, rf_dml_nuisance_final_score = cml_implement(df_EDA, 'RandomForest', 'dml')

In [ ]:
rf_dml_final_result.to_excel('rf_dml_final_result.xlsx')

rf_dml_final_result

In [ ]:
#Transform the results with proper titles

rf_dml_final_result_adj = pd.read_excel('rf_dml_final_result.xlsx')

rf_dml_final_result_adj = rf_dml_final_result_adj.rename(columns={'Unnamed: 0':'Feature', 'coef':'Coefficient',
                                                                  'std err':'Std. Error', 'P>|t|': 'P-value',
                                                                  '2.5 %':'CI Lower Bound (2.5%)', '97.5 %':'CI Upper Bound (97.5%)'}).drop('t', axis=1)

for col in rf_dml_final_result_adj.columns:
    if col != 'Feature':
        if col != 'P-value':
            rf_dml_final_result_adj[col] = rf_dml_final_result_adj[col].round(2)

        else:
            rf_dml_final_result_adj[col] = rf_dml_final_result_adj[col].round(3)
        
rf_dml_final_result_adj['Method'] = 'DML - RF'

rf_dml_final_result_adj

### Causal Forest - Local Centering

In [ ]:
#For transforming the final results of econml file with proper titles

def econml_file(excel, method):
    final_result_adj = pd.read_excel(excel)

    for col in [17, 18]:
        final_result_adj.loc[col, 'mean_point'] = final_result_adj.loc[col,'point_estimate']
        final_result_adj.loc[col, 'stderr_mean'] = final_result_adj.loc[col, 'stderr']
        final_result_adj.loc[col, 'ci_mean_lower'] = final_result_adj.loc[col, 'ci_lower']
        final_result_adj.loc[col, 'ci_mean_upper'] = final_result_adj.loc[col, 'ci_upper']

    final_result_adj.iloc[17, 0] = 'RegulatoryIndex_Low Impact'
    final_result_adj.iloc[18, 0] = 'NumWholesaleWarehouses_None/Few'    

    final_result_adj = final_result_adj.rename(columns={'feature':'Feature', 
                                                        'mean_point':'Coefficient',
                                                        'stderr_mean':'Std. Error',
                                                        'pvalue': 'P-value',
                                                        'ci_mean_lower':'CI Lower Bound (2.5%)', 
                                                        'ci_mean_upper':'CI Upper Bound (97.5%)'})

    final_result_adj = final_result_adj.drop(final_result_adj.iloc[:,7:].columns.tolist()+['zstat'], axis=1)

    for col in final_result_adj.columns:
        if col != 'Feature':
            if col != 'P-value':
                final_result_adj[col] = final_result_adj[col].round(2)

            else:
                final_result_adj[col] = final_result_adj[col].round(3)

    final_result_adj['Method'] = method

    return final_result_adj

In [ ]:
#Product cf output for a single feature

def cf_wrapper(df, ml_type, data_type, column, sensitivity=False):
    
    #parameters for tuning (lasso & logistic)
    Cs = 0.0001*np.logspace(0, 4, 10)
    
    #parameters for tuning (random forest)
    max_features = ['sqrt', 'log2']
    max_depth = [3, 5, 7, 9]
    min_samples_split = [10, 15, 20]
    min_samples_leaf = [5, 10, 15]
    n_estimators = np.arange(500,1000,100)
    
    df_processed = sc_poly_ohe_preprocess(df, ml_type, column)
    
    result_all = pd.DataFrame()
    score_dict = {'yhat_that': {}, 'ml_y': {}, 'ml_t': {}}
    
    #Get the required columns for CF
    Y = df_processed[target_feature1].values.ravel()
    T = df_processed[column].values.ravel()
    X = df_processed.iloc[:, 2:].values
    
    if ml_type == 'Lasso':
        if data_type == 'num':
            est_nonparam = CausalForestDML(model_y=LassoCV(max_iter=10000, random_state=22),
                                           model_t=LassoCV(max_iter=10000, random_state=22), 
                                           cv=5, random_state=22)
        
        else:
            est_nonparam = CausalForestDML(model_y=LassoCV(max_iter=10000, random_state=22),
                                           model_t=LogisticRegressionCV(penalty='l1', solver='liblinear', 
                                                                        Cs=Cs, max_iter=10000, random_state=22), 
                                           discrete_treatment=True, cv=5, random_state=22)
        
        
    else:
        first_stage_reg = lambda: GridSearchCV(estimator=RandomForestRegressor(random_state=22),
                                                      param_grid={
                                                          'max_depth': max_depth,
                                                          'max_features': max_features,
                                                          'min_samples_split': min_samples_split,
                                                          'min_samples_leaf': min_samples_leaf,
                                                          'n_estimators': n_estimators
                                                      }, n_jobs=-1, scoring='neg_mean_squared_error'
                                                     )

        first_stage_class = lambda: GridSearchCV(estimator=RandomForestClassifier(random_state=22),
                                                      param_grid={
                                                          'max_depth': max_depth,
                                                          'max_features': max_features,
                                                          'min_samples_split': min_samples_split,
                                                          'min_samples_leaf': min_samples_leaf,
                                                          'n_estimators': n_estimators
                                                      }, n_jobs=-1, scoring='neg_mean_squared_error'
                                                     )
        
        if data_type == 'num':
            model_y = clone(first_stage_reg().fit(X, Y).best_estimator_)
            model_t = clone(first_stage_reg().fit(X, T).best_estimator_)

            est_nonparam = CausalForestDML(model_y=model_y, model_t=model_t, n_estimators=1000, cv=5, random_state=22)
            
        else:
            est_nonparam = CausalForestDML(model_y=first_stage_reg(), model_t=first_stage_class(), 
                                           discrete_treatment=True, n_estimators=1000, cv=5, random_state=22)
    
    np.random.seed(22)
    est_nonparam.tune(Y, T, X=X, W=None)
       
    if sensitivity == True:
        est_nonparam = est_nonparam.dowhy.fit(Y=Y, T=T, X=X, W=None,
                                              outcome_names=target_feature1, 
                                              treatment_names=[column],
                                              feature_names=df_processed.iloc[:, 2:].columns.tolist()
                                             )
    
    else:
        est_nonparam.fit(Y, T, X=X, W=None)
       
    if data_type == 'num':
        result = est_nonparam.ate_inference(X, T0=0, T1=1).summary().tables[0].as_html()
    
    else:
        result = est_nonparam.summary().tables[0].as_html()
        
    result_df = pd.read_html(result, header=0)[0]
    result_df['feature'] = column
    result_df = result_df.set_index('feature')
    result_all = pd.concat([result_all, result_df])
    
    score_dict['yhat_that'][column] = est_nonparam.score_
    score_dict['ml_y'][column] = est_nonparam.nuisance_scores_y
    score_dict['ml_t'][column] = est_nonparam.nuisance_scores_t
    
    return (result_all, score_dict, est_nonparam)

#### Lasso

In [ ]:
lasso_cf_final_result, lasso_cf_nuisance_final_score = cml_implement(df_EDA, 'Lasso', 'cf')

In [ ]:
lasso_cf_final_result.to_excel('lasso_cf_final_result.xlsx')

lasso_cf_final_result

In [ ]:
lasso_cf_final_result_adj = econml_file('lasso_cf_final_result.xlsx', 'CF - Lasso')

lasso_cf_final_result_adj

#### Random Forest

In [ ]:
rf_cf_final_result, rf_cf_nuisance_final_score = cml_implement(df_EDA, 'RandomForest', 'cf')

In [ ]:
rf_cf_final_result.to_excel('rf_cf_final_result.xlsx')

rf_cf_final_result

In [ ]:
rf_cf_final_result_adj = econml_file('rf_cf_final_result.xlsx', 'CF - RF')

rf_cf_final_result_adj

### Results

#### Model Comparison

In [ ]:
#Get all the results with pvalue less than 0.05

cml_all_results = pd.concat([ols_result_df,
                             lasso_dml_final_result_adj, rf_dml_final_result_adj,
                             lasso_cf_final_result_adj, rf_cf_final_result_adj
                            ]) 

sig_features = set(cml_all_results[(cml_all_results['P-value'] < 0.05) &
                                   (cml_all_results['Feature'].isin(main_numeric_features + main_binary_categorical_features+
                                                                    ['RegulatoryIndex_Low Impact','NumWholesaleWarehouses_None/Few']))]['Feature'])
methods = cml_all_results['Method'].unique()

cml_all_results[(cml_all_results['P-value'] < 0.05) &
                (cml_all_results['Feature'].isin(main_numeric_features + main_binary_categorical_features+
                                                 ['RegulatoryIndex_Low Impact','NumWholesaleWarehouses_None/Few']))]

In [ ]:
#Change the confidence interval for drafting error bar format

cml_all_results_err = cml_all_results.copy()

cml_all_results_err['CI Lower Bound (2.5%)'] = cml_all_results_err['Coefficient'] - cml_all_results_err['CI Lower Bound (2.5%)']
cml_all_results_err['CI Upper Bound (97.5%)'] = cml_all_results_err['CI Upper Bound (97.5%)'] - cml_all_results_err['Coefficient']

cml_all_results_err

In [ ]:
#Helper function for drawing graph with all the significant treatments

def treatment_CI_comp_all(treatment, title, savename):
    palette = sns.color_palette('Set2',len(treatment))
    feature_color_dict = dict(zip(treatment, palette))

    fig, ax = plt.subplots(figsize=(35, 35))

    for i, m in enumerate(methods):
        for j, f in enumerate(treatment):

            row = cml_all_results_err[(cml_all_results_err['Method'] == m) & (cml_all_results_err['Feature'] == f)]
            x = i + j * 0.055  # Slight offset for better visibility
            y = row['Coefficient'].values[0]
            yerr = np.array([[row['CI Lower Bound (2.5%)'].values[0]], [row['CI Upper Bound (97.5%)'].values[0]]])

            ax.errorbar(x, y, yerr=yerr, fmt="o", markersize=20, capsize=10, capthick=8,
                        elinewidth=8, color=feature_color_dict[f], label=f)

    xticks_centers = np.arange(len(methods)) + (len(treatment) - 1) * 0.055 / 2
    ax.set_xticks(xticks_centers)    
    ax.set_xticklabels(methods)
    ax.set_xlabel('Method')
    ax.set_ylabel('ATE with CI')
    ax.set_title(title)

    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), loc='upper right', prop={'size':23}, title = 'Treatment')

    plt.axhline(y=0, color='black', linestyle='--', linewidth=3.5, alpha=0.5)

    plt.rcParams.update({'font.size': 30})
    
    plt.savefig(savename, bbox_inches='tight')
    plt.show()

In [ ]:
#Plot only the features that are significant in at least 2 methods (OLS sig features excluded)

sig_features_count = Counter(cml_all_results_err[(cml_all_results_err['P-value'] < 0.05) & 
                                                 (cml_all_results_err['Method'] != 'OLS')]['Feature'])

sig_features_atleast2 = list({k: v for k, v in sig_features_count.items() if v >= 2}.keys())

treatment_CI_comp_all(sig_features_atleast2, 
                     'Error Bars of Each Method for Each Significant Treatment\n (At least two Occurrences in all Methods - OLS Excluded)',
                     'SigTreatErrorAtleast2.png')

In [ ]:
def treatment_CI_comp_3(treatment, title, methods, savename):
    palette = sns.color_palette('Set2',len(treatment))
    feature_color_dict = dict(zip(treatment, palette))

    fig, ax = plt.subplots(figsize=(20, 20))

    for i, m in enumerate(methods):
        for j, f in enumerate(treatment):

            row = cml_all_results_err[(cml_all_results_err['Method'] == m) & (cml_all_results_err['Feature'] == f)]
            x = i + j * 0.055 # Slight offset for better visibility
            y = row['Coefficient'].values[0]
            yerr = np.array([[row['CI Lower Bound (2.5%)'].values[0]], [row['CI Upper Bound (97.5%)'].values[0]]])

            ax.errorbar(x, y, yerr=yerr, fmt="o", markersize=15, capsize=8, capthick=5,
                        elinewidth=5, color=feature_color_dict[f], label=f)

    xticks_centers = np.arange(len(methods)) + (len(treatment) - 1) * 0.055 / 2
    ax.set_xticks(xticks_centers)    
    ax.set_xticklabels(methods)
    ax.set_xlabel('Method')
    ax.set_ylabel('ATE with CI')
    ax.set_title(title)

    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), loc='upper right', prop={'size':15}, title = 'Treatment')

    plt.axhline(y=0, color='black', linestyle='--', linewidth=2.5, alpha=0.5)

    plt.rcParams.update({'font.size': 20})
    
    plt.savefig(savename, bbox_inches='tight')
    plt.show()

In [ ]:
#Plot only the features that are significant in at least 2 methods (OLS excluded from graph)

methods_noOLS = cml_all_results[cml_all_results['Method'] != 'OLS']['Method'].unique()

treatment_CI_comp_3(sig_features_atleast2, 
                    'Error Bars of Each Method for Each Significant Treatment - OLS excluded\n (At least two Occurrences in all Methods)',
                    methods_noOLS, "SigTreatErrorAtleast2_noOLS.png")

#### CF - Lasso

In [ ]:
#Helper function for building graph for all the coefficient sizes for CF - Lasso

def treatment_effect(df, title, palette, savename, legend=False):
    fig, ax = plt.subplots(figsize = (20, 20))
    sns.barplot(y=df['Coefficient'], x=df['Feature'], palette=palette)

    colors = [bar.get_facecolor() for bar in ax.patches]
    yerr = np.array([df['CI Lower Bound (2.5%)'].values, df['CI Upper Bound (97.5%)'].values])
    for i in range(len(df)):
        ax.errorbar(i, df['Coefficient'].values[i],
                    yerr=[[df['CI Lower Bound (2.5%)'].values[i]], 
                          [df['CI Upper Bound (97.5%)'].values[i]]],
                    fmt='none', color=colors[i], capsize=4, capthick=2, elinewidth=2)

    ax.axhline(y=0, color='black', linestyle='--', linewidth=2.5, alpha=0.5)

    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    ax.set_xlabel('Treatment')
    ax.set_ylabel('ATE with CI')
    ax.set_title(title)
    
    if legend == True:
        legend_elements = [plt.Line2D([0], [0], color=palette[0], lw=10, label='Significant'),
                           plt.Line2D([0], [0], color=palette[-1], lw=10, label='Not Significant')]

        plt.legend(handles=legend_elements, loc = 'upper right', fontsize=20)

    plt.savefig(savename, bbox_inches='tight')
    plt.tight_layout()

In [ ]:
#Show treatment effect by pvalue for CF - Lasso

cf_rf_err_pvalue = cml_all_results_err[cml_all_results_err['Method'] == 'CF - Lasso'].sort_values(by='P-value').reset_index(drop=True)

feature_color_dict = {}

num_sig_features = len(cf_rf_err_pvalue[cf_rf_err_pvalue['P-value'] < 0.05])

palette = sns.color_palette('Set2', num_sig_features)
for i in range(len(cf_rf_err_pvalue)):
    if i < num_sig_features:
        feature_color_dict[cf_rf_err_pvalue['Feature'][i]] = palette[0]
    else:
        feature_color_dict[cf_rf_err_pvalue['Feature'][i]] = palette[1]

treatment_effect(cf_rf_err_pvalue, 'Treatment Effect by P-value', 
                 [feature_color_dict[f] for f in cf_rf_err_pvalue['Feature']],  'cflasso_pvalue.png', legend=True)
plt.axvline(x=len(cf_rf_err_pvalue[cf_rf_err_pvalue['P-value'] < 0.05])-0.5,
            color='black', linestyle='--', linewidth=2.5, alpha=0.5)

plt.savefig('cflasso_pvalue.png', bbox_inches='tight')

### Robustness Checking

In [ ]:
#For fitting the CF - Lasso model again but with also dowhy environment for robustness check

def cat_sensitivity_prep(df, ml_type, column):
    
    df_processed = sc_poly_ohe_preprocess(df, ml_type, column)
    
    if column in main_binary_categorical_features:
        df_processed[column] = df_processed[column].cat.codes.astype('bool')

    model = CausalModel(
        data=df_processed,
        treatment=column,
        outcome=target_feature[0],
        common_causes=df_processed.iloc[:, 2:].columns.tolist(),
        effect_modifiers=df_processed.iloc[:, 2:].columns.tolist()
    )

    identified_estimand = model.identify_effect(
        proceed_when_unidentifiable=True,
        method_name='maximal-adjustment')
    
    print(identified_estimand)
    
    #parameters for tuning logistic regression
    Cs = 0.0001*np.logspace(0, 4, 10)
    
    
    np.random.seed(22)    
    if column in main_binary_categorical_features:        
        cf_lasso_estimate_dw = model.estimate_effect(identified_estimand,
                                                        method_name="backdoor.econml.dml.CausalForestDML",
                                                        control_value=0,
                                                        treatment_value=1,
                                                        target_units="ate",
                                                        confidence_intervals=True,  
                                                        method_params={
                                                            'init_params': {'model_y': LassoCV(max_iter=10000, random_state=22),
                                                                            'model_t': LogisticRegressionCV(penalty='l1', solver='liblinear', 
                                                                                       Cs=Cs, max_iter=10000, random_state=22), 
                                                                            "discrete_treatment":True},
                                                            'fit_params': {}
                                                        }
                                                       )
    
    else:
        cf_lasso_estimate_dw = model.estimate_effect(identified_estimand,
                                                        method_name="backdoor.econml.dml.CausalForestDML",
                                                        control_value=0,
                                                        treatment_value=1,
                                                        target_units="ate",
                                                        confidence_intervals=True,  
                                                        method_params={
                                                            'init_params': {'model_y': LassoCV(max_iter=10000, random_state=22),
                                                                            'model_t': LassoCV(max_iter=10000, random_state=22)},
                                                            'fit_params': {}
                                                        }
                                                       )

    print(cf_lasso_estimate_dw)
    
    return (cf_lasso_estimate_dw, model, identified_estimand)

In [ ]:
#RegulatoryIndex dowhy & CF - Lasso

cf_lasso_dw_reg, dag_reg, estimand_reg = cat_sensitivity_prep(df_EDA, 'Lasso', 'RegulatoryIndex')

In [ ]:
#Warehouse dowhy & CF - Lasso

cf_lasso_dw_ware, dag_ware, estimand_ware = cat_sensitivity_prep(df_EDA, 'Lasso', 'NumWholesaleWarehouses')

#### Random Common Cause (CF - Lasso)

In [ ]:
#Regulatory Index
res_random_reg = dag_reg.refute_estimate(estimand_reg, cf_lasso_dw_reg, 
                                       method_name="random_common_cause",
                                       show_progress_bar=True, random_state=22, n_jobs=-1)

print(res_random_reg)
print('-'*100)

#Warehouse
res_random_ware = dag_ware.refute_estimate(estimand_ware, cf_lasso_dw_ware, 
                                        method_name="random_common_cause",
                                        show_progress_bar=True, random_state=22, n_jobs=-1)

print(res_random_ware)

#### Placebo Treatment (CF - Lasso)

In [ ]:
#Regulatory Index
res_placebo_reg = dag_reg.refute_estimate(estimand_reg, cf_lasso_dw_reg, 
                                             method_name="placebo_treatment_refuter",  
                                             placebo_type="permute",
                                             show_progress_bar=True, random_state=22, n_jobs=-1)

print(res_placebo_reg)
print('-'*100)

#Warehouse
res_placebo_ware = dag_ware.refute_estimate(estimand_ware, cf_lasso_dw_ware, 
                                             method_name="placebo_treatment_refuter",  
                                             placebo_type="permute",
                                             show_progress_bar=True, random_state=22, n_jobs=-1)

print(res_placebo_ware)

#### Data Subset Validation (CF - Lasso)

In [ ]:
#Regulatory Index
res_subset_reg = dag_reg.refute_estimate(estimand_reg, cf_lasso_dw_reg, 
                                             method_name="data_subset_refuter",  
                                             subset_fraction=0.8,
                                             show_progress_bar=True, random_state=22, n_jobs=-1)

print(res_subset_reg)
print('-'*100)

#Warehouse
res_subset_ware = dag_ware.refute_estimate(estimand_ware, cf_lasso_dw_ware, 
                                             method_name="data_subset_refuter",  
                                             subset_fraction=0.8,
                                             show_progress_bar=True, random_state=22, n_jobs=-1)

print(res_subset_ware)

#### Sensitivity Analysis (DML - Lasso)

In [ ]:
#For building the sensitivity analysis table

def dml_sensitivity(df, ml_type, data_type, column):
    
    _, _, model = dml_wrapper(df, ml_type, data_type, column)
    
    ss_df = pd.DataFrame()
    
    model.sensitivity_analysis()
    print(model.sensitivity_summary)    

    ss_df['ci_lower'] = model.sensitivity_params['ci']['lower']
    ss_df['theta_lower'] = model.sensitivity_params['theta']['lower']
    ss_df['theta'] = model.coef
    ss_df['theta_upper'] = model.sensitivity_params['theta']['upper']
    ss_df['ci_upper'] = model.sensitivity_params['ci']['upper']
    ss_df['rv'] = model.sensitivity_params['rv']
    ss_df['rva'] = model.sensitivity_params['rva']
    ss_df['feature'] = column
    ss_df = ss_df.set_index('feature')
    
    model.sensitivity_plot()

    return (ss_df, model)

In [ ]:
#DML - Lasso regulatory index with sensitivity analysis

ss_df, dml_model = dml_sensitivity(df_EDA, 'Lasso', 'cat', 'RegulatoryIndex')

In [ ]:
dml_model.sensitivity_plot()

In [ ]:
#DML - Lasso number of warehouses with sensitivity analysis

ss_df_ware, dml_model_ware = dml_sensitivity(df_EDA, 'Lasso', 'cat', 'NumWholesaleWarehouses')

In [ ]:
dml_model_ware.sensitivity_plot()